In [1]:
file_path = "catkira/1876954_7680KSPS_srsRAN_Project_gnb_short.sigmf-data"
x = read!(file_path, Array{Int32}(undef, 2200544));

In [2]:
𝑓ₛ = 7.68e6
# This should be the nominal RF frequency of the subcarrier that we use as DC for demodulation
# (used for NR phase compensation)
f_dc = 1.87695e9


1.87695e9

In [3]:
Tu = 512
Ncp = div(144, 4)
Ncpl = div(144 + 16, 4)

40

In [4]:
cfo = 18.88  # Hz
ϕ₀ = -2.97  # radians
sfo = -3.2e-6  # parts per one
sto_fine = 12.4e-9  # seconds

1.24e-8

In [5]:
Nsymbols = 1877


1877

In [6]:
# Amplitude adjustment to get symbols of unity amplitude
x = (x[1:2:end] + im * x[2:2:end]) / 2^31 * 24.7; # @show x[1,:];
# Correct for CFO and phase offset
x .*= exp.(-1im * (2 * π * cfo/𝑓ₛ * (1:length(x)) .+ ϕ₀)); # @show x[1,:];

In [7]:
j = sqrt(Complex(-1))

0.0 + 1.0im

In [8]:
Nssblocks = Nsymbols ÷ 140  # Integer division
a₀ = 70420

70420

In [9]:
# phase correction to apply to symbols 0, 1, 2, 3, 4, 5, 6 in a 0.5 ms interval
p₀ = (f_dc .* (0:6) .* (Tu + Ncp)) / 𝑓ₛ
p₁ = p₀ .% 1
phase_corr = exp.(1im * 2 * π * p₁); # @show phase_corr;

In [10]:
using FFTW

In [11]:
Nsymbols = round(Int, (length(x) - a₀) / (1e-3 * 𝑓ₛ) * 14); @show Nsymbols;
# Nsymbols = Int((length(x) - a₀) / (1e-3 * 𝑓ₛ) * 14)
symbols = Array{ComplexF64}(undef, Nsymbols, Tu)
t_symbols = Vector{Float64}(undef, Nsymbols)
for j in 1:Nsymbols
    symidx = j + 1  # +1 since Julia is 1-based
    off =  a₀ + (j-1) * (Ncp + Tu) + div(symidx, 7) * (Ncpl - Ncp)
    off_err = (off - a₀) * sfo + sto_fine * 𝑓ₛ
    off_err_int = round(Int, off_err)
    off_err_frac = off_err - off_err_int
    off_corr = off + off_err_int
    f = fftshift(fft(x[off_corr:off_corr+Tu-1]))  # adjust for 1-based indexing
    f .*= phase_corr[(symidx % 7) + 1]  # use (symidx % 7) + 1 for 1-based indexing
    f .*= exp.(1im * 2 * π * fftshift(fftfreq(Tu)) * Ncp / 2)
    f .*= exp.(1im * 2 * π * fftshift(fftfreq(Tu)) * off_err_frac)
    symbols[j, :] = f
    t_symbols[j] = (off_corr + off_err_frac + Ncp / 2) / 𝑓ₛ
end

Nsymbols = 1877


In [36]:
@show Nsymbols, typeof(symbols), size(symbols), symbols[1:1 , 1:4];

(Nsymbols, typeof(symbols), size(symbols), symbols[1:1, 1:4]) = (1877, Matrix{ComplexF64}, (1877, 512), ComplexF64[-0.0025270452536871376 - 0.003357772173313823im -0.009105439186187447 + 0.0013624020879221348im -0.0038931589703578864 + 0.0023944838627563006im -0.007553634377524973 - 0.010065971837480737im])


In [64]:
for j in 1:Nssblocks
    Δᵢ = 1
    s0 = (j-Δᵢ) * 140
    pbch = vcat(
        symbols[s0 + 1 + Δᵢ, Tu ÷ 2 - 120 + Δᵢ:Tu ÷ 2 + 120],
        symbols[s0 + 2 + Δᵢ, Tu ÷ 2 - 120 + Δᵢ:Tu ÷ 2 - 120 + 48],
        symbols[s0 + 2 + Δᵢ, Tu ÷ 2 - 120 + 192 + Δᵢ:Tu ÷ 2 + 120],
        symbols[s0 + 3 + Δᵢ, Tu ÷ 2 - 120 + Δᵢ:Tu ÷ 2 + 120]
    );
    @show (j)
    @show (s0 + 1 + Δᵢ);@show (Tu ÷ 2 - 120 + Δᵢ);@show (Tu ÷ 2 + 120 + Δᵢ);
    @show (s0 + 2 + Δᵢ);@show (Tu ÷ 2 - 120 + Δᵢ);@show (Tu ÷ 2 - 120 + 48 + Δᵢ);
    @show symbols[s0 + 1 + Δᵢ, Tu ÷ 2 - 120 + Δᵢ:Tu ÷ 2 + 120 + Δᵢ][1]
    @show (pbch[1]), size(pbch);

end

j = 1
s0 + 1 + Δᵢ = 2
(Tu ÷ 2 - 120) + Δᵢ = 137
Tu ÷ 2 + 120 + Δᵢ = 377
s0 + 2 + Δᵢ = 3
(Tu ÷ 2 - 120) + Δᵢ = 137
(Tu ÷ 2 - 120) + 48 + Δᵢ = 185
(symbols[s0 + 1 + Δᵢ, (Tu ÷ 2 - 120) + Δᵢ:Tu ÷ 2 + 120 + Δᵢ])[1] = 0.6677867515280825 + 0.7397103950004779im
(pbch[1], size(pbch)) = (0.6677867515280825 + 0.7397103950004779im, (576,))
j = 2
s0 + 1 + Δᵢ = 142
(Tu ÷ 2 - 120) + Δᵢ = 137
Tu ÷ 2 + 120 + Δᵢ = 377
s0 + 2 + Δᵢ = 143
(Tu ÷ 2 - 120) + Δᵢ = 137
(Tu ÷ 2 - 120) + 48 + Δᵢ = 185
(symbols[s0 + 1 + Δᵢ, (Tu ÷ 2 - 120) + Δᵢ:Tu ÷ 2 + 120 + Δᵢ])[1] = -0.6898121118629664 - 0.7257102087327159im
(pbch[1], size(pbch)) = (-0.6898121118629664 - 0.7257102087327159im, (576,))
j = 3
s0 + 1 + Δᵢ = 282
(Tu ÷ 2 - 120) + Δᵢ = 137
Tu ÷ 2 + 120 + Δᵢ = 377
s0 + 2 + Δᵢ = 283
(Tu ÷ 2 - 120) + Δᵢ = 137
(Tu ÷ 2 - 120) + 48 + Δᵢ = 185
(symbols[s0 + 1 + Δᵢ, (Tu ÷ 2 - 120) + Δᵢ:Tu ÷ 2 + 120 + Δᵢ])[1] = -0.6823994751166962 - 0.6971049539940846im
(pbch[1], size(pbch)) = (-0.6823994751166962 - 0.6971049539940846im, (576,)